In [2]:
from typing import Dict, Any

def merge_alpha_params(param1: Dict[str, Any], param2: Dict[str, Any]) -> Dict[str, Any]:
    """
    Merge two alpha parameter dictionaries by taking non-N/A values from either param
    
    Args:
        param1: First parameter dictionary
        param2: Second parameter dictionary
        
    Returns:
        Merged parameter dictionary
    """
    merged = {}
    
    # List of fields to merge
    fields = ["from_date", "to_date", "product", "product_detail", "level", "user"]
    
    for field in fields:
        # Take value from param1 if not N/A, otherwise from param2
        if param1.get(field) not in [None, "N/A"]:
            merged[field] = param1[field]
        elif param2.get(field) not in [None, "N/A"]:
            merged[field] = param2[field]
        else:
            merged[field] = None
            
    return merged


In [3]:
import requests
import json
from typing import Dict, Any


def llm(
    user_prompt: str,
    system_prompt: str,
    format_schema: Dict[str, Any] = None,
    api: str = 'https://ollama.selab.edu.vn',
    endpoint: str = '/api/chat',
    model: str = "qwen2.5:14b"
) -> Dict[str, str]:
    
    headers = {
      "Content-Type": "application/json"
    }

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]


    if endpoint == '/api/chat' and format_schema is not None:
        data = {
            "model": model,
            "messages": messages,
            "format": format_schema,
            "stream": False
        }
    
    elif endpoint == '/api/generate':
        data = {
            "model": model,
            "prompt": messages,
            "option": {
              "temperature": 0.4  
            },
            "stream": False
        }

    url = f'{api}{endpoint}'
    
    response = requests.post(url, headers=headers, json=data, timeout=600)


    if response.status_code != 200:
        raise Exception(f"Request failed with status {response.status_code}: {response.text}")

    response_data = response.json()
    
    return response_data

In [22]:
def confirm_user_intent(query):
    user_prompt = f"""
    
    # ***User's query***
    {query}

    # ***Example Scenarios:***
    
    - ***User***: "I want to confirm it"
    - ***Assistant***: {{"is_confirmed": 1}}
    
    - ***User***: "Yes, delete it."
    - ***Assistant***: {{"is_confirmed": 1}}

    - ***User***: "No, I changed my mind."
    - ***Assistant***: {{"is_confirmed": 0}}

    - ***User***: "Yes, that's correct."
    - ***Assistant***: {{"is_confirmed": 1}}

    - ***User***: "No, I meant for the last week."
    - ***Assistant***: {{"is_confirmed": 0}}
    """
    return user_prompt


In [23]:
def confirmation_agent(
    query: str
) -> Dict[str, str]:
    
    system_prompt = """
    You are agent detecting the confirmation from user's query, you need to consistently detect and recognize whether user confirms or not.
    """
    
    user_prompt = confirm_user_intent(query)
    
    format_schema = {
        "type": "object",
        "properties": {
            "is_confirmed": {
                "type": "integer"
            }
        },
        "required": [
            "is_confirmed"
        ]
    }
    
    response = llm(
        user_prompt=user_prompt,
        system_prompt=system_prompt,
        format_schema=format_schema,
        endpoint='/api/chat'
    )
    
    return response['message']['content']

In [28]:
response = confirmation_agent(
    query='I am so scared, I am not sure'
)

In [29]:
json.loads(response)

{'is_confirmed': 0}

In [8]:
def detect_task_intent(query):
    
    user_prompt = f"""
    
    # ***User's query***
    {query}

    
    # **Key Guidelines:**
    - Match the user's query to one of the predefined task types listed below.
    - If the query does not match any task type, respond with "Task type not recognized."
    - Provide a concise summary of the detected task type for confirmation.
    - Respond in a polite and professional tone, maintaining a focus on accuracy and helpfulness.

    # ***Predefined Task Types:***
    - Winlost Report
    - Turnover Report
    - Bet Count Report
    - Net Turnover Report
    - Gross Commission
    - Member Report
    - Agent Report
    - Master Report
    - Super Report
    - Company Report
    - Reward (USD) Report
    - Customer / Username Report
    - Outstanding Report
    - Statement Report
    - Create a new customer information (account/member/agent/master/super)
    - Bet List Management
    - Bet Forecasting
    - Transfers
    - Risk Management

    # ***Example Scenarios:***
    - ***User***: "I want to get winlost and turnover report."
    - ***Assistant***: ["Winlost Report", "Turnover Report"]

    - ***User***: "Generate a member and agent report."
    - ***Assistant***: ["Member Report", "Agent Report"]

    - ***User***: "I need a statement and risk management report."
    - ***Assistant***: ["Statement Report", "Risk Management"]
    """
    
    return user_prompt

In [9]:
def task_detection_agent(
    query: str
) -> Dict[str, str]:
    
    system_prompt = """
    You are a task detection agent responsible for identifying and categorizing the type of task requested by the user. Your primary goal is to determine whether the user's query matches one of the predefined task types and confirm the detected task type explicitly.
    """
    
    user_prompt = detect_task_intent(query)
    
    format_schema = {
        "type": "object",
        "properties": {
            "tasks": {
                "type": "array",
                "items": {
                    "type": "string"
                }
            }
        },
        "required": [
            "tasks"
        ]
    }
    
    response = llm(
        user_prompt=user_prompt,
        system_prompt=system_prompt,
        format_schema=format_schema,
        endpoint='/api/chat'
    )
    
    return json.loads(response['message']['content'])

In [13]:
response = task_detection_agent(
    query='I want to get Winlost Report please'
)

In [14]:
response

{'tasks': ['Winlost Report']}

In [32]:
print("Sure, I can help with that. To generate a Winlost Report for day 10 (which would be April 10th, 2025), we need to confirm the following details:\n\n- **Start Date**: `from_date` will be set as \"2025-04-10\".\n- **End Date**: `to_date` will also be set as \"2025-04-10\".\n- **Level**: The level can be selected from the options provided. If you're not sure, we'll use \"All\" for now.\n- **Product**: Similar to the level, if a specific product is required or preferred, please let me know; otherwise, I will use \"All\".\n- **User**: If this report is requested for a particular user or all users under a certain hierarchy, specify the user ID here.\n\nCould you confirm these details? Specifically:\n1. Are there any particular levels or products you want to focus on?\n2. Do you need the report for a specific user or should it include data from all users?\n\nOnce I have confirmation, I'll proceed with generating the Winlost Report.")

Sure, I can help with that. To generate a Winlost Report for day 10 (which would be April 10th, 2025), we need to confirm the following details:

- **Start Date**: `from_date` will be set as "2025-04-10".
- **End Date**: `to_date` will also be set as "2025-04-10".
- **Level**: The level can be selected from the options provided. If you're not sure, we'll use "All" for now.
- **Product**: Similar to the level, if a specific product is required or preferred, please let me know; otherwise, I will use "All".
- **User**: If this report is requested for a particular user or all users under a certain hierarchy, specify the user ID here.

Could you confirm these details? Specifically:
1. Are there any particular levels or products you want to focus on?
2. Do you need the report for a specific user or should it include data from all users?

Once I have confirmation, I'll proceed with generating the Winlost Report.
